# Inspect decaying trajectories

In [ ]:
import numpy as np
import os

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

Select folder where decay trajectories (.pkl files) are located and location of rest file (containing equilibrium position)

In [ ]:
diamond_dir = "../../../soft-robot-control/examples/diamond"
decayData_dir = os.path.join(diamond_dir, "dataCollection/origin2")
rest_file = os.path.join(diamond_dir, 'rest.pkl')

Load all trajectories

In [ ]:
output_node = 1354
t_in = 2
t_out = 5

oData = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out)
Data = {}
Data['oData'] = oData

Plot decay data in (x, y, z)

In [ ]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = []
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)